In [1]:
from dotenv import load_dotenv
import serpapi
import os, requests
from youtube_transcript_api import YouTubeTranscriptApi
from google import genai
from google.genai import types
from datetime import date
from groq import Groq
import json

load_dotenv()

True

In [8]:
client = Groq()

messages=[
        {"role": "system", "content": """
                                You are an AI that answers user queries accurately. Don't ask questions, use what ever resources you have.
                                If a user request benefits from real-world video transcripts, consider using YouTube transcripts for additional context and return the resource used. 
                                If no relevant transcripts are found, proceed with the best available information and only use tools in tool calls.
                                """},
        {"role": "user", "content": "Tell me about Jimmy Carter."}
    ]

In [9]:
def change_youtube_link_to_video_id(link: str):
    return link.replace("https://www.youtube.com/watch?v=", "")

def get_related_youtube_searches(query):
    client = serpapi.Client(api_key=os.getenv('SERPAPI_API_KEY'))

    results = client.search({
        'search_query': query,
        'engine': 'youtube'
    })

    youtube_links = [(video['title'], video['link'], change_youtube_link_to_video_id(video['link'])) for video in results['video_results']]
    return youtube_links[:1]

def format_transcripts(video_data: tuple[str, str]):
    ytt_api = YouTubeTranscriptApi()

    youtube_transcripts = []
    for title, link, video_id in video_data:
        transcripts = ytt_api.fetch(video_id)
        transcript_text = "\n".join([transcript.text for transcript in transcripts])

        youtube_transcripts.append(f'''Title: {title}\nLink: {link}\nTranscript:\n {transcript_text}\n''')
    
    return youtube_transcripts

def get_youtube_transcripts(query: str) -> str:
    ''' returns relevant youtube transcripts of facts'''
    video_data = get_related_youtube_searches(query)

    transcripts = format_transcripts(video_data)

    return "\n\n".join(transcripts) 

In [10]:
transcripts = get_youtube_transcripts('Jimmy Carter')

In [11]:
transcripts

"Title: The unlikely life story of Jimmy Carter\nLink: https://www.youtube.com/watch?v=PNpP5cRmUaw\nTranscript:\n they honored late President Jimmy\xa0\nCarter in his home state of Georgia\nyesterday a motorcade with Carter's flag draped\xa0\ncasket traveled through his hometown of plains and\xa0\xa0\nthen onto Atlanta where he Now lies in Repose at\xa0\nthe Carter presidential center before ceremonies\xa0\xa0\nmove on to Washington on Tuesday we've asked Rita\xa0\nbraver to take a closer look at the unlikely life\xa0\xa0\nstory of our 39th president I would not tell a\xa0\nlie I would not mislead the American people and\xa0\xa0\nI would not betray your trust if anything Define\xa0\nJimmy carter it was his basic decency and morality\xa0\xa0\nthank you again please thank you everybody\xa0\nelected in 1976 in the aftermath of Watergate\xa0\xa0\nand Gerald Ford's controversial pardon of Richard\xa0\nNixon Carter believed he had a responsibility to\xa0\xa0\nrestore Americans' faith in the 

In [12]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_youtube_transcripts",
            "description": "Fetches YouTube transcripts for a given topic.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The topic of interest for which transcripts should be retrieved."
                    }
                },
                "required": ["query"]
            }
        }
    }
]

In [13]:
response = client.chat.completions.create(
    model="llama3-8b-8192",  # Change model if needed
    messages=messages,
    tools=tools,  # Register tool functions
    tool_choice="auto",  # Let Llama decide when to call
    max_tokens=4096
)

BadRequestError: Error code: 400 - {'error': {'message': "Failed to call a function. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': '<tool-use>\n{\n  "tool_calls": [\n    {\n      "id": "pending",\n      "type": "function",\n      "function": {\n        "name": "get_biography"\n      },\n      "parameters": {\n        "name": "Jimmy Carter"\n      }\n    }\n  ]\n}\n</tool-use>\n\nThe 39th President of the United States, Jimmy Carter, served from 1977 to 1981. He was the first and so far the only one-term President of the United States since Franklin D. Roosevelt. Carter was a decorated naval officer, businessman, and politician who was known for his commitment to human rights and his efforts to broker peace between Israel and Egypt. He was awarded the Nobel Peace Prize in 2002 for his efforts to promote peace and human rights. Carter has remained active in politics and humanitarian 

In [ ]:
response_message = response.choices[0].message
tool_calls = response_message.tool_calls

if tool_calls:
    available_functions = {
        'get_youtube_transcripts': get_youtube_transcripts
    }

messages.append(response_message)

for tool_call in tool_calls:
    function_name = tool_call.function.name
    function_to_call = available_functions.get(function_name, None)
    function_args = json.loads(tool_call.function.arguments)
    function_response = function_to_call(**function_args)
    messages.append({
        'tool_call_id': tool_call.id,
        'role': 'tool',
        'name': function_name,
        'content': str(function_response)
    })

second_response = client.chat.completions.create(
    model="llama3-8b-8192",  # Change model if needed
    messages=messages
)

print(second_response.choices[0].message.content)

BadRequestError: Error code: 400 - {'error': {'message': "'messages.3' : for 'role:tool' the following must be satisfied[('messages.3' : property 'tool_to_call_id' is unsupported, did you mean 'tool_call_id'?)]", 'type': 'invalid_request_error'}}

In [ ]:
system_instruction = """
You are an AI that answers user queries accurately. Don't ask questions, use what ever resources you have.
If a user request benefits from real-world video transcripts, consider fetching YouTube transcripts for additional context andreturn the resource used. 
If no relevant transcripts are found, proceed with the best available information.
"""

config = {
    "tools": [
        {
            "function_declarations": [
                {
                    "name": "get_youtube_transcripts",
                    "description": "Fetches YouTube transcripts for a given topic",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "query": {
                                "type": "string",
                                "description": "The topic of interest for which transcripts should be retrieved.",
                            }
                        },
                        "required": ["query"]
                    }
                }
            ]
        }
    ],
    'system_instruction': system_instruction
}

client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

chat = client.chats.create(
    model="gemini-1.5-flash",
    config=config
)

response = chat.send_message(
    "Give me comprehensive information about Jimmy carter"
)

print(response.text)

I need to fetch external resources to provide comprehensive information about Jimmy Carter.  Unfortunately, I cannot access external websites or specific files like YouTube transcripts at this time due to limitations in my current capabilities.  Therefore, I cannot provide a fully comprehensive answer.  To get a complete picture of Jimmy Carter's life and career, I recommend searching online using a search engine like Google, DuckDuckGo, or Bing.  You will find numerous biographies, articles, and documentaries dedicated to his presidency and post-presidency work.



In [ ]:
response = chat.send_message(
    "Use the tool you have"
)

In [ ]:
print(response.text)

I cannot use the `get_youtube_transcripts` function because it is defined but does not have an implementation.  It's a stub.  To get information about Jimmy Carter, I need a functional version of this API or access to other online resources, which I currently do not have.



In [ ]:
functions = [
    {
        "name": "get_youtube_transcripts",
        "description": "Perform a Google search using SerpAPI and return results",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {"type": "string", "description": "Search query"},
                "num_results": {"type": "integer", "description": "Number of results to return"}
            },
            "required": ["query"]
        }
    }
]